In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
with open('poems.txt', 'r', encoding = 'utf-8') as f:
    poems = f.read().strip().split('\n')
print(len(poems))
for i in range(3):
    print(poems[i])

8970
а где тут руль спросил гагарин деревня буркнул королёв ещё спроси а где тут вожжи ещё поехали скажи
вот позитивный анатолий был послан нахуй но сумел вернуться бодрым отдохнувшим и два магнитика привез
перед оксаной в тёмном парке олег распахивает плащ под ним второй а дальше третий оксана терпеливо ждёт


In [5]:
data = pd.Series(poems)
data

0       а где тут руль спросил гагарин деревня буркнул...
1       вот позитивный анатолий был послан нахуй но су...
2       перед оксаной в тёмном парке олег распахивает ...
3       а любишь ролевые игры тебе кого изобразить изо...
4       сперва жена моя сбежала потом собака и коты се...
                              ...                        
8965    сейчас начало девяностых любе ещо почти постпа...
8966    впихнули в тесную в обойму пинком погнали по с...
8967    однажды я сидел на крыше а ты сидела на трубе ...
8968    скорей оденьтесь зинаида я вас обрадовать хочу...
8969    зеркал там не было конечно зато висел портрет ...
Length: 8970, dtype: object

In [6]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w', encoding="utf-8")
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + '\n***\n'
    f.write(data)

In [7]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [8]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=64)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=64)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

In [10]:
train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

C:\Users\snetkova\Anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [11]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
# model.to(device)

In [12]:
training_args = TrainingArguments(
    output_dir="./gpt2-sv", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=6, # number of training epochs
    per_device_train_batch_size=2, # batch size for training
    per_device_eval_batch_size=2,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
#     eval_accumulation_steps=1,
    )

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [14]:
trainer.train()

***** Running training *****
  Num examples = 3118
  Num Epochs = 6
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9354


Step,Training Loss
500,4.684300
1000,4.584800
1500,4.539700
2000,3.895800
2500,3.887600
3000,3.875600
3500,3.361900
4000,3.217100
4500,3.274300
5000,2.893600


Saving model checkpoint to ./gpt2-sv\checkpoint-800
Configuration saved in ./gpt2-sv\checkpoint-800\config.json
Model weights saved in ./gpt2-sv\checkpoint-800\pytorch_model.bin
Saving model checkpoint to ./gpt2-sv\checkpoint-1600
Configuration saved in ./gpt2-sv\checkpoint-1600\config.json
Model weights saved in ./gpt2-sv\checkpoint-1600\pytorch_model.bin
Saving model checkpoint to ./gpt2-sv\checkpoint-2400
Configuration saved in ./gpt2-sv\checkpoint-2400\config.json
Model weights saved in ./gpt2-sv\checkpoint-2400\pytorch_model.bin
Saving model checkpoint to ./gpt2-sv\checkpoint-3200
Configuration saved in ./gpt2-sv\checkpoint-3200\config.json
Model weights saved in ./gpt2-sv\checkpoint-3200\pytorch_model.bin
Saving model checkpoint to ./gpt2-sv\checkpoint-4000
Configuration saved in ./gpt2-sv\checkpoint-4000\config.json
Model weights saved in ./gpt2-sv\checkpoint-4000\pytorch_model.bin
Saving model checkpoint to ./gpt2-sv\checkpoint-4800
Configuration saved in ./gpt2-sv\checkpoint-4

TrainOutput(global_step=9354, training_loss=3.135846999100903, metrics={'train_runtime': 1927.2976, 'train_samples_per_second': 9.707, 'train_steps_per_second': 4.853, 'total_flos': 611031416832000.0, 'train_loss': 3.135846999100903, 'epoch': 6.0})

In [15]:
trainer.save_model()

Saving model checkpoint to ./gpt2-sv
Configuration saved in ./gpt2-sv\config.json
Model weights saved in ./gpt2-sv\pytorch_model.bin


In [16]:
end_token_id = tokenizer.encode('***')[0]

In [29]:
prefix = "тринадцать капель никотина "
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [36]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    end_token=end_token_id,
    do_sample=False, 
    max_length=size+60, 
    repetition_penalty=10., 
    temperature=1,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


тринадцать капель никотина щиплет у вишни николая два желанья одна ольга другая петр
***
вениамин лежит на асфальте и дождь из канифолью стекает по лицу так незаметно что он уже не мыслит ни о чём
***
олег купил в киоске


In [19]:
# trainer.train()

In [20]:
# trainer.save_model()

In [21]:
# prefix = "тринадцать капель никотина "
# tokens = tokenizer(prefix, return_tensors='pt')
# tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [22]:
# size = tokens['input_ids'].shape[1]
# output = model.generate(
#     **tokens, 
#     end_token=end_token_id,
#     do_sample=False, 
#     max_length=size+50, 
#     repetition_penalty=5., 
#     temperature=0.5,
#     num_beams=10,
# )

# decoded = tokenizer.decode(output[0])
# result = decoded[len(prefix):]
# print(prefix + result)

In [23]:
# trainer.train()
# trainer.save_model()

In [24]:
# prefix = "возможно вы знакомы с ольгой "
# tokens = tokenizer(prefix, return_tensors='pt')
# tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [25]:
# size = tokens['input_ids'].shape[1]
# output = model.generate(
#     **tokens, 
#     end_token=end_token_id,
#     do_sample=False, 
#     max_length=size+50, 
#     repetition_penalty=10., 
#     temperature=10.,
#     num_beams=10,
# )

# decoded = tokenizer.decode(output[0])
# result = decoded[len(prefix):]
# print(prefix + result)

In [26]:
# trainer.train()
# trainer.save_model()

In [27]:
# prefix = "оксана радостно запела "
# tokens = tokenizer(prefix, return_tensors='pt')
# tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [28]:
# size = tokens['input_ids'].shape[1]
# output = model.generate(
#     **tokens, 
#     end_token=end_token_id,
#     do_sample=False, 
#     max_length=size+60, 
#     repetition_penalty=20., 
#     temperature=20.,
#     num_beams=30,
# )

# decoded = tokenizer.decode(output[0])
# result = decoded[len(prefix):]
# print(prefix + result)

Почему-то именно продолжать стишки у модели плохо получается, но следующий стишок она генерит все время про Олега, и каждый раз там шедевр)))